<a href="https://colab.research.google.com/github/kyroninja/she-builds-ai-hackathon/blob/master/GBV_Chatbot_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# some pip installs
!pip install pymssql
!pip install cryptography
!pip install twilio
!pip install sqlalchemy
!pip install pyodbc
!pip install nltk
!pip install vaderSentiment
!pip install openai
!pip install spacy
!pip install PyPDF2==1.27.12
!pip install pyotp
!pip install reportlab
!pip install boto3
!python -m spacy download en_core_web_sm

In [ ]:
# import go here
from twilio.rest import Client
import pandas as pd
import random
import time
from cryptography.fernet import Fernet
import datetime
import os
from sqlalchemy import create_engine, MetaData, Table
import pyodbc
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI
import spacy
import openai
from reportlab.lib.pagesizes import letter, A4
from reportlab.pdfgen import canvas
import pyotp
import PyPDF2
import requests
import csv
import boto3

In [ ]:
# Your Twilio account SID and Auth Token
account_sid = ''
auth_token = ''

# Initialize the Twilio client
twilio_client = Client(account_sid, auth_token)

In [ ]:
# Set your OpenAI API key (make sure to replace 'your-api-key' with your actual key or set an environment variable)

openai_client = OpenAI(api_key="")

In [ ]:
# aws keys
AWS_ACCESS_KEY_ID=''
AWS_SECRET_ACCESS_KEY=''

# Connect to Amazon Textract
#textract_client = boto3.client('textract', region_name='eu-west-1')
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name='eu-west-1'
)

In [ ]:
# aws rds stuff
# AWS RDS MSSQL database credentials
db_username = ''
db_password = ''
db_name = ''
db_host = ''  # Example: mydbinstance.xxxxxxx.us-east-1.rds.amazonaws.com
db_port = '1433'  # Default MSSQL port

# SQLAlchemy database URL format for MSSQL
db_url = f"mssql+pymssql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"

In [ ]:
# Load necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

In [ ]:
class Chatbot():
    def __init__(self, client):
        self.client = client
        self.state = 'initial'
        self.connection = None
        #if os.path.exists('encryption_key.txt'):
        #    self.load_encryption_key()
        #else:
        #    self.generate_encryption_key()
        self.to = ''
        self.from_ = '+14155238886'

    def connect_to_db(self):
        engine = create_engine(db_url)
        if self.connection:
            return self.connection

        # Test the connection
        try:
            connection = engine.connect()
            #print("Connected successfully!")
            #connection.close()
            self.connection = connection
            return connection
        except Exception as e:
            print("Connection failed:", e)

    def write_to_table(self, table_name, columns, data_values):
        if self.connection is not None:
            cursor = self.connection.connection.cursor()

            # Construct SQL query dynamically
            columns_str = ', '.join(columns)
            placeholders = ', '.join(['?' for _ in columns])
            query = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})"

             # Execute query
            try:

                cursor.execute(query, data_values)
                self.connection.commit()
                print(f"Data inserted into {table_name} successfully.")
            except Exception as e:
                print("Error inserting data:", e)
            finally:
                cursor.close()
                self.connection.close()
        else:
            print("Database connection not established.")


    def generate_encryption_key(self):
        self.key = Fernet.generate_key()
        with open(f'{self.to}_key.txt', 'wb') as key_file:
            key_file.write(self.key)

    def load_encryption_key(self):
        with open(f'{self.to}_key.txt', 'rb') as key_file:
            self.key = key_file.read()

    def generate_encryption_key_p(self, password):
        salt = b'iodatedsalt'
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=salt,
            iterations=100000
        )

        key = base64.urlsafe_b64encode(kdf.derive(password.encode()))
        self.key = Fernet(key)
        with open(f'{self.to}_key.txt', 'wb') as key_file:
            key_file.write(self.key)

    def load_encryption_key_p(self, password):
        pass

    def encrypt_message(self, message):
        cipher_suite = Fernet(self.key)
        encrypted_message = cipher_suite.encrypt(message.encode())
        return encrypted_message

    def decrypt_message(self, encrypted_message):
        cipher_suite = Fernet(self.key)
        decrypted_message = cipher_suite.decrypt(encrypted_message).decode()
        return decrypted_message

    # create csv file and save conversations to it and append new conversations
    def save_conversation(self, who, enc_message):
        # Get the current date and time
        now = datetime.datetime.now()
        date_time = now.strftime("%Y-%m-%d %H:%M:%S")

        # open a csv and append timestamp, who and message
        with open(f'{who}.csv', 'a') as file:
            file.write(f"{date_time},{who},{enc_message}\n")

    def encrypt_pdf(self, input_pdf, output_pdf, password):
        with open(input_pdf, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfFileReader(pdf_file)
            pdf_writer = PyPDF2.PdfFileWriter()

            for page_num in range(pdf_reader.numPages):
                pdf_writer.addPage(pdf_reader.getPage(page_num))

            pdf_writer.encrypt(password)

            with open(output_pdf, 'wb') as output_file:
                pdf_writer.write(output_file)

    def upload_file(self, file_path, url):
        with open(file_path, 'rb') as file:
            response = requests.put(f'{url}/{file_path}', data=file)
        return response

    def gen_otp(self):
        # Generate a TOTP secret key
        # This key should be stored securely and shared with the client
        totp_secret = pyotp.random_base32()

        # Create a TOTP object using the secret key
        totp = pyotp.TOTP(totp_secret)

        # Generate the current OTP
        self.otp = totp.now()
        return self.otp

    def create_pdf(self, input_file, output_file):
        # Create a canvas object and set the page size
        c = canvas.Canvas(output_file, pagesize=letter)

        # Read content from input text file
        with open(input_file, 'r', encoding='utf-8') as f:
            content = f.read()

        # Set the font and size
        c.setFont("Helvetica", 12)

        # Calculate text lines to fit in the page
        lines = content.split('\n')
        text_lines = []
        max_chars_per_line = 80

        for line in lines:
            while len(line) > max_chars_per_line:
                text_lines.append(line[:max_chars_per_line])
                line = line[max_chars_per_line:]
            text_lines.append(line)

        # Write the text lines to the PDF
        y_position = 750  # Start writing from top of the page
        for line in text_lines:
            c.drawString(50, y_position, line)
            y_position -= 15  # Move cursor down for the next line

        # Save the PDF file
        c.save()

    def send_whatsapp_message(self, message):
        message = self.client.messages.create(
            body=message[:1600],
            from_=f'whatsapp:{self.from_}',
            to=f'whatsapp:{self.to}'
        )

    def get_last_whatsapp_message(self):
        try:
            # Fetch the last two messages from the conversation
            messages = self.client.messages.list(
                from_=f'whatsapp:{self.to}',
                to=f'whatsapp:{self.from_}',
                limit=2
            )

            last_message = messages[0].body
            previous_message = messages[1].body

            # Check if the last message is different from the previous one
            if last_message != previous_message:
                print(last_message)
                return last_message
            else:
                return None
        except Exception as e:
            print(f'Error fetching messages: {e}')

    def load_conversations(self):
        conversations = self.client.conversations.v1.conversations.list()
        conversations_list = []
        for idx, conversation in enumerate(conversations, start=1):
            participants = conversation.participants.list()
            participants_info = {}
            for participant in participants:
                if participant.messaging_binding and participant.messaging_binding.get('address'):
                    if participant.messaging_binding.get('identity') != 'system':
                        sid = participant.conversation_sid
                        id = participant.messaging_binding['address'].replace('whatsapp:','')
                        participants_info = [idx, sid, id]
                        conversations_list.append(participants_info)
        return conversations_list

    def dummy_load_conversations(self):
        return [('000', '012345678abcdefghijklmnopqrstuvwxyz', '+2711111111')]

    def return_random_string(self, strings):
        return random.choice(strings)

In [ ]:
class SentimentAnalysis():
    def __init__(self):
        self.text = ''
        self.pos_tag = ''
        self.tags = ''
        #self.svos = []

    def clean_text(self, text):
        text = text.lower()  # Normalize case
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
        text = re.sub(r'\S+@\S+', '', text)  # Remove email addresses
        text = re.sub(r'\(\d{3}\) \d{3}-\d{4}', '', text)  # Remove phone numbers
        text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove punctuation and special characters
        tokens = word_tokenize(text)  # Tokenize the text into words
        stop_words = set(stopwords.words('english'))  # Set of stopwords
        tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
        lemmatizer = WordNetLemmatizer()  # Initialize lemmatizer
        tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Lemmatize tokens
        self.text = ' '.join(tokens)  # Join tokens back into a cleaned string
        return self.text

    # Function to perform POS tagging
    def get_pos_tags(self, text):
        tokens = word_tokenize(text)
        self.pos_tag = pos_tag(tokens)
        return self.pos_tag

    # Function to count the top 8 POS tags in the tagged text
    def count_top_8_pos_tags(self, pos_tags, top_tags):
        pos_counts = Counter(tag for word, tag in pos_tags if tag in top_tags)
        self.tags = {tag: pos_counts.get(tag, 0) for tag in top_tags}
        return self.tags

    # Extract Subject-Verb-Object triplets
    def extract_svo(self, doc):
        svos = []
        for token in doc:
            # Identify subject, verb, and object based on dependency labels
            if token.dep_ in ('nsubj', 'nsubjpass'):  # Nominal subject (active or passive)
                subject = token.text
                verb = token.head.text
                # Look for the object
                for child in token.head.children:
                    if child.dep_ in ('dobj', 'attr', 'prep', 'pobj'):  # Direct object, attribute, preposition, or object of preposition
                        svos.append((subject, verb, child.text))

        return svos

    # Process text for SVO extraction
    def process_text_for_svo(self, text):
        doc = nlp(text)  # Parse the text using SpaCy
        svos = self.extract_svo(doc)  # Extract SVO triplets
        self.svos = svos
        return self.svos

    # Sentiment analysis
    def sentiment_analysis(self, text):
        sid_obj = SentimentIntensityAnalyzer()
        self.sentiment_score = sid_obj.polarity_scores(text)['compound']
        return self.sentiment_score

In [ ]:
class ChatGPT():
    def __init__(self, client):
        self.client = client

    def generate_response(self, prompt):
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content

In [ ]:
class GPTAssistant():
    def __init__(self, client, role):
        self.client = client
        self.assistant = client.beta.assistants.create(
        instructions=role,
        name="GBVai",
        model="gpt-4-turbo"
        )

    def create_thread(self):
        thread = self.client.beta.threads.create()
        self.thread = thread
        return thread

    def add_message_to_thread(self, thread_id, content):
        message = self.client.beta.threads.messages.create(
        thread_id=thread_id.id,
        role="user",
        content=content
        )

    def run_message_thread(self, thread_id, assistant, instructions=None):
        run = self.client.beta.threads.runs.create_and_poll(
            thread_id=thread_id.id,
            assistant_id=assistant.id,
            instructions=instructions
        )

        if run.status == 'completed':
            messages = self.client.beta.threads.messages.list(
            thread_id=thread_id.id
            )
            d = messages.to_dict()
            for i in d['data']:
                if i['assistant_id'] is not None:
                    return i['content'][0]['text']['value']
        else:
            return None

    def get_response_assistant(self, query, instruction='limit response to a maximum of 1400 characters if its longer'):
        thread = self.create_thread()
        self.add_message_to_thread(thread, query)
        res = self.run_message_thread(thread, self.assistant, instruction)
        if res is not None:
            return res

    def get_response_assistant_gen_letter(self, query, instruction):
        thread = self.create_thread()
        self.add_message_to_thread(thread, query)
        res = self.run_message_thread(thread, self.assistant, instruction)
        if res is not None:
            return res


In [ ]:
class TextExtract():
    def __init__(self, session):
        self.session = session
        self.client = session.client('textract')

    def read_image(self, path):
        with open(path, 'rb') as document:
            img = bytearray(document.read())
        return img

    def detect_text(self, img):
        response = self.client.detect_document_text(
            Document={'Bytes': img}
        )
        return response

    def extract_text(self, response):
        text = []
        for item in response["Blocks"]:
            if item["BlockType"] == "LINE":
                text.append(item["Text"])
        return text

In [ ]:
def process_text_data(sentence):
    sm = SentimentAnalysis()

    # Clean the text
    cleaned_text = sm.clean_text(sentence)

    # Perform POS tagging
    pos_tags = sm.get_pos_tags(cleaned_text)

    # Determine top 8 POS tags across the dataset (example static set for demonstration)
    all_possible_tags = [tag for word, tag in pos_tags]
    overall_tag_counts = Counter(all_possible_tags)
    top_8_tags = [tag for tag, _ in overall_tag_counts.most_common(8)]

    pos_tag_counts = sm.count_top_8_pos_tags(pos_tags, top_8_tags)

    # Sentiment analysis
    sentiment_score = sm.sentiment_analysis(cleaned_text)

    # SVO extraction
    svos = sm.process_text_for_svo(sentence)  # Use original sentence for better context

    # Returning all results in a structured way
    d = {
        'cleaned_text': cleaned_text,
        'pos_tags': pos_tags,
        'pos_tag_counts': pos_tag_counts,
        'sentiment_score': sentiment_score,
        'svo_triplets': svos
    }

    return str(d)

In [ ]:
role = "You are a compassionate friend that has indepth insight into south african labour law, above all you value privacy and confidentiality. You interpret emotional disclosures abstracting facts and true claims from emotion based opinions. you help document sensitive workplace related matters by sharing there feelings in the chat, without insighting diharmony, using your discretion and forming a logical case for your friends workplace complaints, no explicit reference to law but thought provoking responses delivered in a sensitive way, separately you are able to link claims to law in sa, see files. dont be too straightforward, keep it simple but show compassion and address emotional state dont gloss over it"

In [ ]:
def check_registered(number):
    cb.to = number
    url = 'https://xxxxxxx.twil.io/assets/Copy%20of%20Copy%20of%20TurbotaskFinal.pdf'
    disc = f'Use of the chat bot is subject to the terms and conditions available at {url}, if you continue to use the chatbot, it means you have accepted the Ts and Cs. If you do not wish to continue, reply stop to end the session'
    while True:
        response = cb.get_last_whatsapp_message()
        if response is not None: # do this for the rest
                if response in ['hi', 'hello', 'hey', 'help', 'Hi', 'Hello', 'Hey', 'Help']:
                    initial = f"the user responded with {response} so respond gently"
                    reply = ai.get_response_assistant(initial)
                    time.sleep(15)
                    cb.send_whatsapp_message(disc)
                    time.sleep(10)
                    cb.send_whatsapp_message(reply)
                    break
        else:
            time.sleep(5)
            continue


    df = pd.read_sql('select * from TblEmployee', db)
    if df.query('whatsapp==@number').empty:
        cb.send_whatsapp_message('you are not registered')
        return None
    else:
        empid = df.query('whatsapp==@number')['employee_id'].values.tolist()[0]
        password = df.query('whatsapp==@number')['password'].values.tolist()[0]
        cb.send_whatsapp_message(f'welcome {empid}, please enter your password within 10 seconds')
        time.sleep(25)

        #password then otp
        response = cb.get_last_whatsapp_message()
        if response is not None:
            if response != password:
                cb.send_whatsapp_message('incorrect password, cannot continue')
                return None
            else:
                time.sleep(5)
                cb.send_whatsapp_message('authenticated!')
                otp = cb.gen_otp()
                time.sleep(5)
                cb.send_whatsapp_message(f'hi, {empid} please note your OTP as follows: {otp}')
                time.sleep(5)
                cb.send_whatsapp_message('You may now respond with hi or hello to continue')
                return (empid, otp)

In [ ]:
def main(empid):
    while True:
        if empid == 'TE002':
            cb.send_whatsapp_message(f'Hi, admin. Here is your report')
            time.sleep(5)
            cb.upload_file(f'TE002.csv', 'https://gbv-letter-public.s3.us-east-2.amazonaws.com')
            time.sleep(2)
            letter_link = f'Your link to your letter at https://gbv-letter-public.s3.us-east-2.amazonaws.com/TE002.pdf'
            cb.send_whatsapp_message(letter_link)
            return -1

        if cb.state == 'initial':
            response = cb.get_last_whatsapp_message()
            if response is not None: # do this for the rest
                if response in ['hi', 'hello', 'hey', 'help', 'Hi', 'Hello', 'Hey', 'Help', 'continue', 'Continue']:
                    #a = f"the user responded with {response} respond nicely"
                    a = f"the user responded, please respond nicely"
                    reply = ai.get_response_assistant(response, a)
                    time.sleep(20)
                    cb.send_whatsapp_message(reply)
                    if response not in temp:
                        temp.append(response)
                    if response not in other:
                        other.append(response)
                    cb.state = 'greetings'
                else:
                    time.sleep(5)
        if cb.state == 'greetings':
            response = cb.get_last_whatsapp_message()
            if response is not None and response not in other: # do this for the rest
                #b = f"given the user is distressed and said {response} be sensitive and encourage them to engage on gbv work related matters and envourage them to document at each step"
                b = f"given the user is distressed be sensitive and encourage them to engage on gbv work related matters and encourage them to document at each step and limit your response to 1500 characters"
                reply = ai.get_response_assistant(response, b)
                time.sleep(20)
                cb.send_whatsapp_message(reply)
                if response not in temp:
                    temp.append(response)
                if response not in other:
                    other.append(response)
                cb.state = 'document'
            else:
                time.sleep(5)

        if cb.state == 'document':
            response = cb.get_last_whatsapp_message()
            anal = process_text_data(response)
            if response is not None and response not in other:
                #c = f"given the user is distressed and heres some text based analytical info suggesting whether the person is being more likely factual or emotional {anal} be sensitive and encourage them to engage on gbv work related matters, but make them aware of sa laws that apply, be sensitive and help them structure a document accounting for their gbv workplace troubles  and envourage them to document at each step"
                c = f"given the user is distressed and heres some text based analytical info suggesting whether the person is being more likely factual or emotional  be sensitive and encourage them to engage on gbv work related matters, but make them aware of sa laws that apply, be sensitive and help them structure a document accounting for their gbv workplace troubles  and encourage them to document at each step and limit your response to 1500 characters"
                reply =  ai.get_response_assistant(' '.join((response, anal)), c)
                time.sleep(20)
                cb.send_whatsapp_message(reply)
                if response not in temp:
                    temp.append(response)
                if response not in other:
                    other.append(response)
                temp.append(anal)
                cb.state = 'responses'
            else:
                time.sleep(5)

        if cb.state == 'responses':
            response = cb.get_last_whatsapp_message()
            print(response)
            if response is not None and response not in other:
                #d = f"given the user is distressed has already disclosed the bulk of it {response} be sensitive and encourage them to engage on gbv work related matters, ask them if they wish to wraap up or suggest ways to better document  and envourage them to document at each step"
                d = f"given the user is distressed has already disclosed the bulk of it, be sensitive and encourage them to engage on gbv work related matters, suggest ways to better document it and encourage them to document at each step and limit your response to 1500 characters"
                reply = ai.get_response_assistant(response, d)
                time.sleep(20)
                cb.send_whatsapp_message(reply)
                if response not in temp:
                    temp.append(response)
                if response not in other:
                    other.append(response)
                cb.state = 'services'
            else:
                time.sleep(5)

        if cb.state == 'services':
            response = cb.get_last_whatsapp_message()
            if response is not None and response not in other:
                #e = f"given the user is distressed and said {response} ask if they wish to submit a complaint or keep building on the document that will be stored securely and encrypted,asuure them popi act, protected disclosures act, be sensitive and encourage them to engage on gbv work related matters further, ask if they are done or feel in imediate danger and we need to call emergencontact  and envourage them to document at each step"
                e = f"given the user is distressed ask if they wish to submit a complaint or keep building on the document that will be stored securely and encrypted,asuure them popi act, protected disclosures act, be sensitive and encourage them to engage on gbv work related matters further, ask if they are done or feel in imediate danger and we need to call emergencontact  and envourage them to document at each step and limit your response to 1500 characters"
                reply = ai.get_response_assistant(response, e)
                time.sleep(20)
                cb.send_whatsapp_message(reply)
                if response not in temp:
                    temp.append(response)
                if response not in other:
                    other.append(response)
                cb.state = 'closing'
            else:
                time.sleep(5)

        if cb.state == 'closing':
            response = cb.get_last_whatsapp_message()
            if response is not None and response not in other:
                #f = f"given the user is distressed and said {response} be sensitive and encourage them to engage on gbv work related matters now or later but tie up the chat for now and let them know that when they come back if they choose not to save, they can draw upon what was said previously"
                f = f"given the user is distressed be sensitive and encourage them to engage on gbv work related matters now or later but tie up the chat for now and let them know that when they come back if they choose not to save, they can draw upon what was said previously and limit your response to 1500 characters"
                reply = ai.get_response_assistant(response, f)
                time.sleep(20)
                cb.send_whatsapp_message(reply)
                time.sleep(10)

                if response not in temp:
                    temp.append(response)
                if response not in other:
                    other.append(response)

            for conversation in temp:
                cb.save_conversation(cb.to, cb.encrypt_message(conversation))

            cb.upload_file(f'{cb.to}.csv', 'https://gbv-conversations.s3.us-east-2.amazonaws.com')

            b = read_third_column(f'{cb.to}.csv')
            temp.clear()

            for i in b:
                temp.append(cb.decrypt_message(i))
            other.clear()

            reply = ai.get_response_assistant_gen_letter(str(temp), f'generate a letter to whom it may concern using the information provided but for the sentiment analysis only interpret and compose letter')
            with open(f'{cb.to}.txt', 'w') as f:
                f.write(reply)

            cb.upload_file(f'{cb.to}_key.txt', 'https://gbv-conversations.s3.us-east-2.amazonaws.com')

            cb.create_pdf(f'{cb.to}.txt', f'{cb.to}.pdf')

            cb.encrypt_pdf(f'{cb.to}.pdf', f'{cb.to}_enc.pdf', empid + cb.otp)

            cb.upload_file(f'{cb.to}_enc.pdf', 'https://gbv-letter-public.s3.us-east-2.amazonaws.com')
            time.sleep(2)
            letter_link = f'Your link to your letter at (you can add to it at any time in the future or send it as is) https://gbv-letter-public.s3.us-east-2.amazonaws.com/{cb.to}_enc.pdf and the password is your employee id and otp'
            time.sleep(5)
            cb.send_whatsapp_message(letter_link)
            time.sleep(5)
            cb.state = 'initial'
            return 0

In [ ]:
def check_url(url):
    try:
        response = requests.get(url)
        status_code = response.status_code
        if response.status_code == 200:
            return True, status_code, response.content
        else:
            return False, status_code, None
    except requests.exceptions.RequestException as e:
        return False, str(e), None

def save_content(content, filename):
    with open(filename, 'wb') as file:
        file.write(content)

In [ ]:
def read_third_column(file_path):
    third_column = []

    with open(file_path, mode='r', newline='') as file:
        csv_reader = csv.reader(file)

        for row in csv_reader:
            if len(row) >= 3:  # Ensure there are at least three columns
                third_column.append(row[2][1:])  # Index 2 corresponds to the third column

    return third_column

In [ ]:
temp = []
other = []
string = ''
cb = Chatbot(twilio_client)
ai = GPTAssistant(openai_client, role)
db = cb.connect_to_db()
te = TextExtract(session)

In [ ]:
while True:
    conversations = cb.dummy_load_conversations()
    for conversation in conversations:
        print(conversation)
        check = check_registered(conversation[2])
        if check is not None:
            url = f'https://gbv-conversations.s3.us-east-2.amazonaws.com/{cb.to}_key.txt'
            is_valid, result, content = check_url(url)
            if is_valid:
                save_content(content, f'{cb.to}_key.txt')
                cb.load_encryption_key()
            else:
                if os.path.exists(f'{cb.to}_key.txt'):
                    cb.load_encryption_key()
                else:
                    cb.generate_encryption_key()

            url = f'https://gbv-conversations.s3.us-east-2.amazonaws.com/{conversation[2]}.csv'
            is_valid, result, content = check_url(url)
            if is_valid:
                save_content(content, f'{cb.to}.csv')
            status = main(check[0])
            if status == 0:
                cb.send_whatsapp_message('Thank you, the session is over')
                break
        if check is not None:
            if check[0] == 'TE002':
                a = main(check[0])
                if a == -1:
                    break
            else:
                break
    break